In [58]:
import cv2
import os
import numpy as np 
from IPython.display import Image  # for displaying images
from PIL import Image, ImageDraw, ImageFilter 
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import easyocr
import matplotlib.pyplot as plt
import imutils
import glob
import re
import pytesseract
pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract.exe"
import easyocr
import uuid

In [137]:
def plot_bounding_box(image_file, image, annotation_list, i, save_file_directory):
    annotations = np.array(annotation_list) 
    w, h = image.size 
    directory = re.search('\w+.jpg', image_file)[0]
    transformed_annotations = np.copy(annotations) 
    transformed_annotations[:,[0,2]] = annotations[:,[0,2]] * w 
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * h  
    
    transformed_annotations[:,0] = transformed_annotations[:,0] - (transformed_annotations[:,2] / 2) - 20 # 20 | 80
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2) - 20 # 20 | 80
    transformed_annotations[:,2] = transformed_annotations[:,0] + transformed_annotations[:,2] + 40 # 40 | 160
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3] + 40 # 40 | 160
    
    for ann in transformed_annotations: 
        x0, y0, x1, y1 = ann 
        cropped_image = image.crop((x0,y0, x1, y1))
        filepath = os.path.join(save_file_directory, directory)
        cropped_image.save(filepath  + str(uuid.uuid4()) + '.jpg')

In [138]:
filepath = 'yolov7\\number-detection-3\\train'
k_global = 0
k_temp = k_global
exp_detect_path = filepath
labels_path = os.path.join(exp_detect_path, 'labels')
labels = glob.glob(os.path.join(labels_path, '*.txt'))
print(labels)
for label in labels:
    with open(label, 'r') as f:
        result = list()
        for line in f: 
            annotation_list = line.split("\n")[:-1] 
            annotation_list = [x.split(" ") for x in annotation_list] 
            annotation_list = [[float(y) for y in x if y != ''] for x in annotation_list]
            if len(annotation_list) != 0:
                result.append(annotation_list)
    image_file = label.replace("labels\\", "images\\")
    image_file = image_file[0:len(image_file)-3] 
    image_file += "jpg" 
    assert os.path.exists(image_file) 
    
    #Load the image 
    image = Image.open(image_file) 
    
    #Plot the Bounding Box 
    i = 1
    for bbox in result:
        plot_bounding_box(image_file, image, [bbox[::][0][1::]], i, "calcRecal")
        i += 1
    k_temp += 1

['yolov7\\number-detection-3\\train\\labels\\image-10-53_jpg.rf.482a75c05e94c42092db138578a86a54.txt', 'yolov7\\number-detection-3\\train\\labels\\image-100-6_jpg.rf.ae4cdd6bb174ec94e9a8828bed2aea8b.txt', 'yolov7\\number-detection-3\\train\\labels\\image-102-6_jpg.rf.abd7f59332d22879015192ba2427c15c.txt', 'yolov7\\number-detection-3\\train\\labels\\image-103-90_jpg.rf.eb8a37237f67af983e48a69311cdd230.txt', 'yolov7\\number-detection-3\\train\\labels\\image-104-90_jpg.rf.8b345e71e29b0eb875ff3d442e56db5b.txt', 'yolov7\\number-detection-3\\train\\labels\\image-105-30_jpg.rf.ec03952861975c4994c68e0af649198b.txt', 'yolov7\\number-detection-3\\train\\labels\\image-107-30_jpg.rf.f1a46ac9353e084e9aab07381c9f820a.txt', 'yolov7\\number-detection-3\\train\\labels\\image-108-30_jpg.rf.82f69add76aba73b4234d7414d93122f.txt', 'yolov7\\number-detection-3\\train\\labels\\image-110-30_jpg.rf.fb1d1438d8db6e45d8968d205aa3992e.txt', 'yolov7\\number-detection-3\\train\\labels\\image-111-12_jpg.rf.a66da8e83f4

In [43]:

reader = easyocr.Reader(['en']) 

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [39]:
def get_strongest(files_result):
    for file_result in files_result:
        exists_result_strong = [x for x in file_result if x != '' and len(x) != 0]                
        max_strong = [x for x in exists_result_strong if isinstance(x, (list, tuple))]
        if len(max_strong) != 0:
            max_strong = max(x[-1][-1] for x in max_strong)
        elif len(exists_result_strong) != 0:
            return exists_result_strong[0]
        for x in exists_result_strong:
            if isinstance(x, (list, tuple)): 
                if (x[-1][-1] == max_strong):
                    return x
    return None

In [139]:
files = glob.glob(os.path.join("calcRecal\\", '*.jpg'))
result = list()
for file in files:
    local_result = list()
    img = cv2.imread(file)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    msk = cv2.inRange(hsv, np.array([0, 0, 175]), np.array([179, 255, 255]))
    krn = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 3))
    dlt = cv2.dilate(msk, krn, iterations=1)
    thr = 255 - cv2.bitwise_and(dlt, msk)
    text = pytesseract.image_to_string(thr, lang='eng', config="--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789")
    mass_text = reader.readtext(thr, allowlist='0123456789')
    local_result.append(file)
    local_result.append(text)
    local_result.append(mass_text)
    result.append(local_result)

In [140]:
cnt = 0
for res in result:
    try:
        first = re.search('calcRecal\\\\\d+', res[0])[0]
        true_answer = re.search('\d+', first)[0]
        predict_answer = get_strongest(res[1::])
        #print(str(true_answer) + " " + str(predict_answer))
        if true_answer != predict_answer:
            cnt += 1
    except: 
        print(res)
print(cnt)
print(len(result))

['calcRecal\\139_jpga6d168b2-b641-4755-8e30-62f79d11aa7e.jpg', '', [([[41, 19], [60, 19], [60, 33], [41, 33]], '910', 0.7074867986036817)]]
['calcRecal\\148_jpgea3e4c19-fa9c-411d-ad09-fac3d9f8a2d2.jpg', '', [([[18, 22], [60, 22], [60, 54], [18, 54]], '148', 0.9999960081211735)]]
['calcRecal\\158_jpgeaabd0ce-003d-4090-a94a-11a49cb7e398.jpg', '', [([[10, 9], [47, 9], [47, 23], [10, 23]], '41', 0.04586441973735587), ([[16, 18], [54, 18], [54, 44], [16, 44]], '158', 0.9798267483711243)]]
['calcRecal\\15_jpg28210a89-64e8-469c-be28-4ce4ff2ed74d.jpg', '', [([[18, 22], [42, 22], [42, 56], [18, 56]], '15', 0.9758484297507096)]]
['calcRecal\\17_jpg7342a644-0a06-4faf-8973-804fec1e9672.jpg', '', [([[4, 26], [34, 26], [34, 54], [4, 54]], '0', 0.8642025936373479)]]
['calcRecal\\218_jpg02fc799b-157b-40cd-bd70-fa8b91fbee71.jpg', '', [([[22, 20], [67, 20], [67, 46], [22, 46]], '', 0.0)]]
['calcRecal\\24_jpg53ee5110-ceec-4fc4-afbc-f506b67ebb46.jpg', '', [([[16, 18], [46, 18], [46, 52], [16, 52]], '2', 0

In [131]:
files = glob.glob(os.path.join("calcRecal\\", '*.jpg'))
result = list()
for file in files:
    local_result = list()
    img = cv2.imread(file)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    msk = cv2.inRange(hsv, np.array([0, 0, 175]), np.array([179, 255, 255]))
    krn = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 3))
    dlt = cv2.dilate(msk, krn, iterations=1)
    thr = 255 - cv2.bitwise_and(dlt, msk)
    text = pytesseract.image_to_string(thr, lang='eng', config="--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789")
    mass_text = reader.readtext(img, allowlist='0123456789')
    local_result.append(file)
    local_result.append(text)
    local_result.append(mass_text)
    result.append(local_result)

In [132]:
cnt = 0
for res in result:
    try:
        first = re.search('calcRecal\\\\\d+', res[0])[0]
        true_answer = re.search('\d+', first)[0]
        predict_answer = get_strongest(res[1::])
        #print(str(true_answer) + " " + str(predict_answer))
        if true_answer != predict_answer:
            cnt += 1
    except: 
        print(res)
print(cnt)
print(len(result))

['calcRecal\\101_jpgb6b3a754-b5c9-4a91-b927-9a903b150f65.jpg', '', [([[36, 38], [70, 38], [70, 68], [36, 68]], '4', 0.06526982806296733)]]
['calcRecal\\127_jpg9dc7caf0-3c69-4643-9000-abe5fc87dbb1.jpg', '', [([[38, 40], [68, 40], [68, 70], [38, 70]], '07', 0.1712796054932621), ([[25, 99], [47, 99], [47, 117], [25, 117]], '1121', 0.19313286244869232)]]
['calcRecal\\128_jpg2f3c627c-f37f-4eff-91f6-2a01fc035189.jpg', '', [([[36, 38], [66, 38], [66, 66], [36, 66]], '1', 0.12859179447517377), ([[67, 79], [100, 79], [100, 97], [67, 97]], '870', 0.3780503176195108)]]
['calcRecal\\128_jpgc01f6a5d-f08d-4e6b-b27b-53841d54662c.jpg', '', [([[61, 71], [95, 71], [95, 87], [61, 87]], '081019', 0.11574106828042993)]]
['calcRecal\\129_jpg43c4eeef-676d-470c-af3f-2a0052d73fc4.jpg', '', [([[68, 40], [96, 40], [96, 48], [68, 48]], '56', 0.26609537159436114), ([[36, 40], [64, 40], [64, 68], [36, 68]], '129', 0.8215004857694538)]]
['calcRecal\\12_jpgadd80dc1-43c8-4fce-abb2-bac9f21f40b9.jpg', '', [([[56.8905996

In [141]:
files = glob.glob(os.path.join("calcRecal\\", '*.jpg'))
result = list()
for file in files:
    local_result = list()
    img = cv2.imread(file)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    msk = cv2.inRange(hsv, np.array([0, 0, 175]), np.array([179, 255, 255]))
    krn = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 3))
    dlt = cv2.dilate(msk, krn, iterations=1)
    thr = 255 - cv2.bitwise_and(dlt, msk)
    mass_text = reader.readtext(img, allowlist='0123456789')
    local_result.append(file)
    local_result.append(mass_text)
    result.append(local_result)

In [142]:
cnt = 0
for res in result:
    try:
        first = re.search('calcRecal\\\\\d+', res[0])[0]
        true_answer = re.search('\d+', first)[0]
        predict_answer = get_strongest(res[1::])
        print(true_answer, predict_answer)
        if true_answer != predict_answer:
            cnt += 1
    except: 
        print(res)
print(cnt)
print(len(result))

101 None
['calcRecal\\116_jpg7efcd374-cf5e-4765-b7d8-e6504bc321d1.jpg', [([[18, 20], [48, 20], [48, 46], [18, 46]], '6', 0.9467136698113201)]]
11 None
122 None
127 None
['calcRecal\\127_jpgee28413e-e0e8-47b9-9fc8-e7076aa5638a.jpg', [([[18, 20], [60, 20], [60, 48], [18, 48]], '0', 0.14275246770063177)]]
128 None
['calcRecal\\128_jpgf7638c9d-c734-418e-9b71-db6b9b830e9a.jpg', [([[16, 16], [46, 16], [46, 46], [16, 46]], '173', 0.20741989903920163)]]
128 None
['calcRecal\\129_jpg4fdfe6a4-9f7b-4a7e-95cb-93e6242ee603.jpg', [([[16, 20], [44, 20], [44, 48], [16, 48]], '129', 0.6051709851062687)]]
['calcRecal\\12_jpg88fcf604-b400-4fc9-95fb-a45a88cee8ee.jpg', [([[18, 22], [48, 22], [48, 52], [18, 52]], '12', 0.9760942429704338)]]
12 None
137 None
139 None
139 None
['calcRecal\\148_jpgea3e4c19-fa9c-411d-ad09-fac3d9f8a2d2.jpg', [([[18, 22], [60, 22], [60, 54], [18, 54]], '148', 0.9999991052683532)]]
['calcRecal\\158_jpgeaabd0ce-003d-4090-a94a-11a49cb7e398.jpg', [([[18, 18], [56, 18], [56, 42], [18,

In [143]:
files = glob.glob(os.path.join("calcRecal\\", '*.jpg'))
result = list()
for file in files:
    local_result = list()
    img = cv2.imread(file)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    msk = cv2.inRange(hsv, np.array([0, 0, 175]), np.array([179, 255, 255]))
    krn = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 3))
    dlt = cv2.dilate(msk, krn, iterations=1)
    thr = 255 - cv2.bitwise_and(dlt, msk)
    text = pytesseract.image_to_string(img, lang='eng', config="--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789")
    local_result.append(file)
    local_result.append(text)
    result.append(local_result)
cnt = 0
for res in result:
    try:
        first = re.search('calcRecal\\\\\d+', res[0])[0]
        true_answer = re.search('\d+', first)[0]
        predict_answer = res[1::][0]
        print(true_answer, predict_answer)
        if true_answer != predict_answer:
            cnt += 1
    except: 
        print(res)
print(cnt)
print(len(result))

101 
116 
11 
122 
127 25

127 
128 
128 
128 
129 
12 
12 
137 
139 5

139 
148 1

158 
15 
15 
15 
171 
171 
171 
17 
17 
17 
17 1

205 
20 
218 
218 1

218 
218 9

21 
21 2

248 
24 
24 2

24 
262 
262 
262 
262 92

262 
262 
262 
262 22

266 
266 266

266 88

266 
266 
266 
26 
26 
26 
26 26

26 
26 
26 
27 
27 
27 
27 
27 
27 
284 
28 1

28 
28 
30 
30 
30 
30 
325 
325 
325 
325 
325 325

35 
35 
35 
37 
39 
3 
3 
417 
427 
427 
43 
43 
43 
43 
45 
45 
45 
46 
46 
46 
46 
46 
46 
46 
46 
46 
46 
46 
46 
502 
50 
52 
59 
5 5

5 
5 5

5 5

5 
5 53

5 
67 
67 
67 
6 
720 
753 
77 
81 
81 
837 
83 
83 
83 5

83 3

83 
83 
83 
83 
8 
8 
8 0

8 
8 
8 
8 
8 
8 
8 
8 
8 1

8 8

903 
90 
90 
90 
90 
90 0

90 
90 
90 
90 
92 
94 
95 
96 
96 
99 
99 99

99 8

99 
169
169
